<a href="https://colab.research.google.com/github/Karthick47v2/speech-emotion-classifier/blob/main/feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Load data


In [ ]:
! pip install kaggle

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download uwrfkaggler/ravdess-emotional-speech-audio
! kaggle datasets download ejlok1/surrey-audiovisual-expressed-emotion-savee
! kaggle datasets download ejlok1/toronto-emotional-speech-set-tess

! unzip ravdess-emotional-speech-audio
! unzip surrey-audiovisual-expressed-emotion-savee
! unzip toronto-emotional-speech-set-tess


## Import Libs


In [ ]:
import glob
import pandas as pd
import numpy as np
import random
import librosa
import librosa.display
import matplotlib.pyplot as plt


In [ ]:
# Mapping

# 1 - neutral
# 2 - calm ##
# 3 - happy
# 4 - sad
# 5 - angry
# 6 - fear
# 7 - disgust
# 8 - surprise


## Analyze data

### Load path

In [ ]:
# RAVDESS
r_path_list = glob.glob('/content/Actor_*/**')
r_emotion_list = [int((x.split('.')[0]).split('-')[2]) for x in r_path_list]
r_df = pd.DataFrame({'path': r_path_list, 'emotion': r_emotion_list})

# SAVEES
s_path_list = glob.glob('/content/ALL/**')
s_emotion_list = [(list((x.split('.')[0]).split('_')[1])[0]) + (list((x.split('.')[0]).split('_')[1])
                                                                [1] if list((x.split('.')[0]).split('_')[1])[0] == 's' else '0') for x in s_path_list]
s_df = pd.DataFrame({'path': s_path_list, 'emotion': s_emotion_list})

# TESS
t_path_list = glob.glob(
    '/content/TESS Toronto emotional speech set data/**/**')
t_emotion_list = [((x.split('.')[0]).split('/')[-1].split('_')[-1])
                  for x in t_path_list]
t_df = pd.DataFrame({'path': t_path_list, 'emotion': t_emotion_list})


Drop 'Calm' as we are not interested in it.

In [ ]:
s_df['emotion'] = s_df['emotion'].replace(
    ['a0', 'd0', 'f0', 'h0', 'n0', 'sa', 'su'], [5, 7, 6, 3, 1, 4, 8])
t_df['emotion'] = t_df['emotion'].replace(
    ['angry', 'disgust', 'fear', 'happy', 'neutral', 'ps', 'sad'], [5, 7, 6, 3, 1, 8, 4])


In [ ]:
df = pd.concat([r_df, s_df, t_df], ignore_index=True)
df = df[df['emotion'] != 2]


In [ ]:
df['emotion'].value_counts()


As there are no large different b/w counts we don't need to care about class weight

In [ ]:
wav_len = []

for path in df['path']:
    y, sr = librosa.load(path, sr=None, mono=True)

    wav_len.append(y.shape[0] / sr)

df['length'] = wav_len


In [ ]:
df.groupby(['emotion'])['length'].mean()


In [ ]:
hist = df.hist()


TAKE 4s (4 \* 22050 = 88200)


### Sample

Load audio

In [ ]:
y, sr = librosa.load(df['path'][0], mono=True)


wave

In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.waveshow(y, sr=sr)
plt.title('waveform')
plt.show()


trim silenced time in leading and trailing parts

In [ ]:
trimmed_y, idx = librosa.effects.trim(y, top_db=50)
librosa.display.waveshow(trimmed_y, sr=sr)
plt.title('trimmed waveform')
plt.show()


STFT (Short-Time Fourier Transform) spectrogram

In [ ]:
n_fft = 2048
hop_length = 512

# normalize
normalized_y = librosa.util.normalize(trimmed_y)
# short-term fourier transform
stft = librosa.core.stft(normalized_y, n_fft=n_fft, hop_length=hop_length)
# log scale
db = librosa.amplitude_to_db(abs(stft))

librosa.display.specshow(db, x_axis='time', y_axis='log')
plt.title('STFT')
plt.colorbar()
plt.show()


MFCC (Mel Frequency Cepstral Coefficient) Spectrogram (FEATURE #1)

In [ ]:
# mfcc
mfcc = librosa.feature.mfcc(normalized_y, sr, n_mfcc=13)

librosa.display.specshow(mfcc, x_axis='time', y_axis='mel')
plt.title('MFCC')
plt.colorbar()
plt.show()


Log-MEL Spectrogram (FEATURE #2)

In [ ]:
# mel scaled spectogram
mel = librosa.feature.melspectrogram(S=stft, n_mels=128)

mel_db = librosa.amplitude_to_db(abs(mel))
# mel_db = librosa.util.normalize(mel_db)

librosa.display.specshow(mel_db, x_axis='time', y_axis='mel')
plt.title('Mel-Scaled')
plt.colorbar()
plt.show()


## Data preprocessing

In [ ]:
def load_wav(path, stop):
    # 22.05 kHz, 16 bit
    y, sr = librosa.load(path, mono=True)
    trimmed_y, idx = librosa.effects.trim(y, top_db=50)
    normalized_y = librosa.util.normalize(trimmed_y)
    y_chunks = [normalized_y[i:(i+stop)]
                for i in range(0, len(normalized_y), stop)]
    return y_chunks, sr


In [ ]:
def extract_mfcc(y, sr, n_mfcc, n_fft, hop_length):
    mfcc = librosa.feature.mfcc(
        y=y, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    normalized_mfcc = librosa.util.normalize(mfcc)

    return normalized_mfcc


def extract_mel(y, n_mel, n_fft, hop_length):
    stft = librosa.core.stft(y, n_fft=n_fft, hop_length=hop_length)

    mel = librosa.feature.melspectrogram(S=stft, n_mels=n_mel)
    mel_db = librosa.amplitude_to_db(abs(mel))
    normalized_mel = librosa.util.normalize(mel_db)

    return normalized_mel


In [ ]:
# augmentation - noise

def add_noise(y, s_noise, e_noise):
    noise = random.uniform(s_noise, e_noise)
    noised_y = y + np.random.rand(len(y)) * noise

    return noised_y


In [ ]:
def add_padding(x, padding):
    padded_x = []

    for i in x:
        sh = len(i[0])

        if padding > 0 & sh < padding:
            x = padding - sh
            l_pad = x // 2
            r_pad = x - l_pad
            i = np.pad(i, pad_width=((0, 0), (l_pad, r_pad)), mode='constant')

        padded_x.append(i)

    return padded_x


- n_fft = 2048, hop_length = 512 gave high accuracy with relatively small model..
- we divide audio into 4s chunks and process (we will get 2 data from 4 < x <= 8 audio clip)
- max_frame for this config is, 173 frames. Which needs to be noted as we need to preprocess our data for prediction
- 13 MFCC coefficients and 128 filter banks for Log-Mel taken to this study as it would be more than enough.


In [ ]:
mfcc = []
mel = []
label = []
max_frame = 0

n_fft = 2048
hop_length = 512

add_aug = True

for idx, row in df.iterrows():
    y_chunks, sr = load_wav(row['path'], 88200)

    for y in y_chunks:

        if len(y) >= n_fft:

            ex_mfcc = extract_mfcc(y, sr, 13, n_fft, hop_length)
            ex_mel = extract_mel(y, 128, n_fft, hop_length)

            mfcc.append(ex_mfcc)
            mel.append(ex_mel)
            label.append(row['emotion'])

            if add_aug:
                noised_y = add_noise(y, 0.005, 0.02)

                aug_mfcc = extract_mfcc(noised_y, sr, 13, n_fft, hop_length)
                aug_mel = extract_mel(noised_y, 128, n_fft, hop_length)

                mfcc.append(aug_mfcc)
                mel.append(aug_mel)
                label.append(row['emotion'])

            if max_frame < ex_mfcc.shape[1]:
                max_frame = ex_mfcc.shape[1]

padded_mfcc = add_padding(mfcc, max_frame)
padded_mel = add_padding(mel, max_frame)


### Check data

In [ ]:
np.unique(label, return_counts=True)


In [ ]:
librosa.display.specshow(padded_mel[1000], x_axis='time', y_axis='mel')
plt.title('Mel-Scaled')
plt.colorbar()
plt.show()


In [ ]:
librosa.display.specshow(padded_mfcc[0], x_axis='time', y_axis='mel')
plt.title('Mel-Scaled')
plt.colorbar()
plt.show()


## Save 

In [ ]:
X_mfcc = np.array(padded_mfcc)
X_mel = np.array(padded_mel)
y = np.array(label)

np.save('X_mfcc_2048', X_mfcc)
np.save('X_mel_2048', X_mel)
np.save('y_2048', y)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
! mv X_mel_2048.npy X_mfcc_2048.npy y_2048.npy drive/MyDrive/SER
